In [31]:
import pandas as pd
import snf
import numpy as np
from sklearn import cluster
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.cluster import HDBSCAN
from matplotlib.pyplot import cm
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import silhouette_score
from tableone import TableOne
import plotly.graph_objects as go
import plotly.subplots as sp

In [32]:
def calculate_clustering_scores(X, labels):
    if len(set(labels)) > 1: 
        ch_score = calinski_harabasz_score(X, labels)
        db_score = davies_bouldin_score(X, labels)
        sil_score = silhouette_score(X, labels)
        return ch_score, db_score, sil_score
    else:
        return np.nan, np.nan, np.nan  


In [33]:
metrics = ['cosine'] # euclidean
dim_red_list = ['pca', 'tsne']
dims = list(range(2,44))
types_input = [['cytokines', 'genera']] #['cytokines'], ['genera'],
K = [5,10,15,20]

In [34]:
genera = pd.read_csv('./dati/test/genera.csv', header = 0, index_col='id')
cyto = pd.read_csv('./dati/test/cytokines.csv', header = 0, index_col='id')
clinical_original     = pd.read_csv('./dati/test/clinical_carmen.csv', header = 0, index_col='id')
names = clinical_original["id_microbio"]
cyto = cyto.drop([4, 23, 34])


clinical = clinical_original.copy()
clinical['age_category'] = clinical['age_molecular'].apply(lambda x: 1 if x < 60 else 0)
label_encoder = LabelEncoder()
clinical_encoded = clinical.copy()

# for col in clinical.columns:
#     clinical_encoded[col] = label_encoder.fit_transform(clinical[col].astype(str))
# normalizer_clinical = MinMaxScaler(feature_range=(0,1))
# normalizer_clinical.fit(clinical_encoded)
# clinical_norm = pd.DataFrame(normalizer_clinical.transform(clinical_encoded.copy()), columns=clinical_encoded.columns)

normalizer_genera = MinMaxScaler(feature_range=(0,1))
normalizer_genera.fit(genera)
genera_norm = pd.DataFrame(normalizer_genera.transform(genera.copy()), columns=genera.columns)
genera_norm = genera / 2*genera.sum()
np.fill_diagonal(genera_norm.values, 0.5)

normalizer_cyto = MinMaxScaler(feature_range=(0,1))
normalizer_cyto.fit(cyto)
cyto_norm = pd.DataFrame(normalizer_cyto.transform(cyto.copy()), columns=cyto.columns)
cyto_norm = cyto / 2*cyto.sum()
np.fill_diagonal(cyto_norm.values, 0.5)

In [35]:
results = []

for type_input in types_input:
    for metric in metrics:
        for dim_red in dim_red_list:
            for dim in dims:
                for k in K:
                
                    # TODO: wrap in a function
                    affinities = []
                    for data in type_input:
                        # if data == "clinical_norm":
                        #     data_array = clinical_norm.astype(float).values
                        #     affinity = snf.make_affinity(data_array, metric="jaccard")
                        if data == "genera":
                            data_array = genera.astype(float).values
                            affinity = snf.make_affinity(data_array, metric=metric, K=k)
                        if data == "cytokines":
                            data_array = cyto.astype(float).values
                            affinity = snf.make_affinity(data_array, metric=metric, K=k)
                        affinities.append(affinity)

                    if len(type_input) > 1:
                        fused = snf.snf(affinities, K=k)
                        type_input_str = type_input[0] + ', ' + type_input[1]
                    else:
                        fused = affinity
                        type_input_str = type_input[0] 
                    
                    if dim_red == 'pca':
                        pca = PCA(n_components=dim)
                        data = pca.fit_transform(fused)
                    elif dim_red == "tsne":
                        tsne = TSNE(n_components=dim, random_state=42, method='exact')
                        data = tsne.fit_transform(fused)

                    hdb = HDBSCAN(min_cluster_size=3, min_samples=3, alpha=1.0, cluster_selection_method="leaf", cluster_selection_epsilon=0.1)
                    hdb.fit(data)
                    labels = hdb.labels_

                    # here plot
                    # df = pd.DataFrame(data)
                    # df['Cluster'] = labels
                    # df = df.set_index(names)

                    ch_score, db_score, sil_score = calculate_clustering_scores(data, labels)

                    results.append([dim_red, dim, f'K={str(k)}', sil_score])


In [36]:
results_df = pd.DataFrame(results, columns=['Dimensionality reduction technique', 'Dimensions', 'K', 'Silhouette'])
# pd.set_option('display.max_rows', None)
results_df.to_csv("./output/model_optimization.csv")
results_df

,Dimensionality reduction technique,Dimensions,K,Silhouette
0,pca,2,K=5,0.179773
1,pca,2,K=10,0.313217
2,pca,2,K=15,0.360578
3,pca,2,K=20,0.421039
4,pca,3,K=5,0.281661
...,...,...,...,...
331,tsne,42,K=20,NaN
332,tsne,43,K=5,-0.449714
333,tsne,43,K=10,NaN
334,tsne,43,K=15,NaN


In [44]:
scores = ["Silhouette"] #"Calinski-Harabasz", ,
# type_inputs = "Fused Layers"  #results_df['type_input'].unique()
type_inputs = results_df["K"].unique()
techniques = results_df['Dimensionality reduction technique'].unique()


# Create a function to generate the plots
def create_plots(df, scores, type_inputs, techniques):
    for score in scores:
        # Create a subplot figure with 1 row and 3 columns (one for each type_input)
        fig = sp.make_subplots(rows=2, cols=2, subplot_titles=type_inputs,  horizontal_spacing=0.05,  # Decrease horizontal spacing
            vertical_spacing=0.1)
        
        # Loop over each type_input to generate subplots
        for idx, type_input in enumerate(type_inputs):
            df_subset = results_df[results_df['K'] == type_input]

            # Add traces for each technique
            for technique in techniques:
                df_technique = df_subset[df_subset['Dimensionality reduction technique'] == technique]

                # print(len(df_technique['Dimensions']), len(df_technique[score]))
                row = (idx // 2) + 1
                col = (idx % 2) + 1

                # Plot the data for this technique and input type
                fig.add_trace(go.Scatter(
                    x=df_technique['Dimensions'],
                    y=df_technique[score],
                    mode='lines+markers',
                    name=technique,
                    legendgroup=technique,
                ), row=row, col=col)

                fig.update_yaxes(range=[-1, 1], row=row, col=col)

            # Update the layout of the figure
            fig.update_layout(
                title=f'{score} Score by embedding dimension and by number of k-nearest neighbours',
                xaxis_title='Dimensions',
                yaxis_title=score,
                height=1200,
                width=1200,
                showlegend=True
            )

        # # Set aspect ratio to 1 for each subplot
        # for i in range(1, len(type_inputs) + 1):
        #     fig.update_xaxes(scaleanchor=f'y{i}', scaleratio=1, row=(i // 2) + 1, col=(i % 2) + 1)
        #     fig.update_yaxes(scaleanchor=f'x{i}', scaleratio=1, row=(i // 2) + 1, col=(i % 2) + 1)

        # Show the figure
        fig.show()

# Call the function with the correct parameters
create_plots(results_df, scores, type_inputs, techniques)